In [1]:
import mysql.connector
import numpy as np
import pandas as pd
import time

import orderdic as od

In [2]:
host = 'localhost'
user = 'root'
passwd = 'pass'
dbname = 'orderdb'
table_name = 'orders'

# cursor.execute(f"TRUNCATE {table_name};")

In [3]:
def getdb(pending_pdf_colname):
    cnx = mysql.connector.connect(host=host, user=user, password=passwd)
    cursor = cnx.cursor()
    cursor.execute(f"USE {dbname};")
    query = f"SELECT {', '.join(pending_pdf_colname)} " + "FROM orders WHERE pending = 1"
    cursor.execute(query)
    pending = cursor.fetchall()
    pending = pd.DataFrame(pending, columns=pending_pdf_colname)
    return pending

In [4]:
pending_pdf_colname = ['id', 'red', 'blue', 'green', 'orderdate', 'address']

In [5]:
pdf = getdb(pending_pdf_colname)

In [6]:
pdf.head()

,id,red,blue,green,orderdate,address
0,1,5,2,10,2020-01-26 17:19:48,2
1,2,5,8,3,2020-01-26 17:19:49,6
2,3,5,6,2,2020-01-26 17:19:50,4
3,4,9,2,1,2020-01-26 17:19:51,6
4,5,7,4,7,2020-01-26 17:19:52,1


In [7]:
from orderdic import *

In [8]:
orders = [makeOrder(row) for _, row in pdf.iterrows()]

In [12]:
def count_items(orders):
    if type(orders) == dict:
        return sum(orders['item'].values())
    else:
        return sum([count_items(order) for order in orders])

In [14]:
count_items(orders[0:5])

76

In [109]:
pdf = pdf[pdf['item'] <= 20] # 일단 partial order 로 쪼개는게 구현 안됐으니까

In [82]:
orders = [dict(row) for idx, row in pdf.iterrows()]

In [22]:
orders

[{'id': 1,
  'address': 2,
  'item': {'r': 5, 'g': 10, 'b': 2},
  'orderdate': Timestamp('2020-01-26 17:19:48')},
 {'id': 2,
  'address': 6,
  'item': {'r': 5, 'g': 3, 'b': 8},
  'orderdate': Timestamp('2020-01-26 17:19:49')},
 {'id': 3,
  'address': 4,
  'item': {'r': 5, 'g': 2, 'b': 6},
  'orderdate': Timestamp('2020-01-26 17:19:50')},
 {'id': 4,
  'address': 6,
  'item': {'r': 9, 'g': 1, 'b': 2},
  'orderdate': Timestamp('2020-01-26 17:19:51')},
 {'id': 5,
  'address': 1,
  'item': {'r': 7, 'g': 7, 'b': 4},
  'orderdate': Timestamp('2020-01-26 17:19:52')},
 {'id': 6,
  'address': 2,
  'item': {'r': 1, 'g': 8, 'b': 2},
  'orderdate': Timestamp('2020-01-26 17:19:53')},
 {'id': 7,
  'address': 3,
  'item': {'r': 6, 'g': 2, 'b': 5},
  'orderdate': Timestamp('2020-01-26 17:19:54')},
 {'id': 8,
  'address': 5,
  'item': {'r': 8, 'g': 7, 'b': 9},
  'orderdate': Timestamp('2020-01-26 17:19:55')},
 {'id': 9,
  'address': 2,
  'item': {'r': 10, 'g': 4, 'b': 2},
  'orderdate': Timestamp('2020-

In [116]:
driving_time

array([[0.02188562, 1.03035649, 2.05600391, 3.0041803 , 3.00375058,
        2.07748455, 1.04750056],
       [1.03477355, 0.00653805, 1.05630519, 2.06842966, 3.04417691,
        3.03433386, 2.03273778],
       [2.07336875, 1.07712379, 0.03142375, 1.05161302, 2.19266441,
        3.04502235, 3.03420437],
       [3.03927124, 2.14943544, 1.04868659, 0.04614379, 1.08454   ,
        2.01185274, 3.04853681],
       [3.07702347, 3.18221721, 2.01867046, 1.16916733, 0.05144994,
        1.11614361, 2.11615342],
       [2.11234546, 3.18111659, 3.13578868, 2.05699566, 1.06517044,
        0.07484987, 1.07404974],
       [1.1801119 , 2.18314699, 3.06430067, 3.12755504, 2.1537611 ,
        1.21700853, 0.01033574]])

In [85]:
def evaluate_order(curr_location, order):
    destination = order['address']
    driving_cost = driving_time[curr_location][destination]
    num_items = order['red'] + order['green'] + order['blue']
    unloading_cost = num_items
    if num_items == 0:
        order['value'] = 0
    else:
        order['value'] = 1 / (unloading_cost + driving_cost)
    return order

In [86]:
orders = [evaluate_order(0, order) for order in orders]

In [95]:
orders = sorted(orders, key=lambda x: -x['value'])

In [97]:
def count_items(orders):
    if type(orders) == dict:
        return orders['red'] + orders['green'] + orders['blue']        
    else:
        return sum([order['red'] + order['green'] + order['blue'] for order in orders])

In [98]:
def split_orders(orders, item_limit=20):
    i = 0
    order_group = []
    while count_items(order_group) <= item_limit:
        i += 1
        order_group = orders[:i]
    return orders[:i-1], orders[i-1:]

In [99]:
def split_all_orders(orders):
    scheduled_orders = [orders]
    while count_items(scheduled_orders[-1]) > 20:
        split_order = split_orders(scheduled_orders[-1])
        scheduled_orders.pop()
        scheduled_orders += split_order
    return scheduled_orders

In [15]:
from collections import defaultdict

In [24]:
def group_same_address(partial_orders):
    partial_by_address = defaultdict(list)
    for order in partial_orders:
        partial_by_address[order['address']].append(order)
    return dict(partial_by_address)

In [1]:
from greedy_scheduler import stringify_path, make_path_command

In [3]:
make_path_command(robot_loc=2, robot_orient = 'cw', address=[4,6,3,2,1])

'216439'

In [48]:
a.insert(1,1)

In [41]:
a

[1, 1, 2, 3]

In [29]:
a += group_same_address(orders[:10]).values()

In [31]:
a[0]

[{'id': 1,
  'address': 2,
  'item': {'r': 5, 'g': 10, 'b': 2},
  'orderdate': Timestamp('2020-01-26 17:19:48')},
 {'id': 6,
  'address': 2,
  'item': {'r': 1, 'g': 8, 'b': 2},
  'orderdate': Timestamp('2020-01-26 17:19:53')},
 {'id': 9,
  'address': 2,
  'item': {'r': 10, 'g': 4, 'b': 2},
  'orderdate': Timestamp('2020-01-26 17:19:56')}]

In [105]:
order_group = split_all_orders(orders)

In [107]:
order_group[-1]

[{'id': 29,
  'red': 10,
  'blue': 6,
  'green': 4,
  'orderdate': Timestamp('2020-01-26 17:20:16'),
  'address': 4,
  'item': 20,
  'value': 0.04323064674735045},
 {'id': 92,
  'red': 0,
  'blue': 0,
  'green': 0,
  'orderdate': Timestamp('2020-01-26 17:24:29'),
  'address': 6,
  'item': 0,
  'value': 0}]

In [ ]:
def schedule(pending_df, curr_location):
    # Load driving time
    address = [0, 1, 2, 3, 4, 5, 6]
    driving_time = np.zeros((7, 7))
    for start_add in address:
        driving_time[start_add] = np.array([time_generator(start_add, end_add) 
                                            if end_add != start_add else 0.0
                                            for end_add in address])
    order_group = []
    evaluated_orders = [evaluate_order(dict(row)) for idx, row in pdf.iterrows()]
    sorted_by_value = sorted(evaluated_orders, key=lambda x: x['value'], ascending=False)
    

## Splitting orders

In [126]:
pdf = getdb(pending_pdf_colname)

In [127]:
pdf['item'] = pdf['red'] + pdf['blue'] + pdf['green']

In [128]:
need_split = pdf[pdf['item'] > 20]

In [150]:
need_split

,id,red,blue,green,orderdate,address,item
7,8,8,9,7,2020-01-26 17:19:55,5,24
12,13,8,10,8,2020-01-26 17:20:00,6,26
15,16,2,10,9,2020-01-26 17:20:03,5,21
18,19,3,8,10,2020-01-26 17:20:06,4,21
22,23,9,9,10,2020-01-26 17:20:10,4,28
123,124,22,18,4,2020-01-28 18:47:04,5,44
124,125,22,4,7,2020-01-28 18:47:05,3,33
125,126,6,25,12,2020-01-28 18:47:06,5,43
126,127,16,25,9,2020-01-28 18:47:07,5,50
127,128,18,14,24,2020-01-28 18:47:08,6,56


In [151]:
orders = [dict(row) for idx, row in need_split.iterrows()]

In [152]:
orders

[{'id': 8,
  'red': 8,
  'blue': 9,
  'green': 7,
  'orderdate': Timestamp('2020-01-26 17:19:55'),
  'address': 5,
  'item': 24},
 {'id': 13,
  'red': 8,
  'blue': 10,
  'green': 8,
  'orderdate': Timestamp('2020-01-26 17:20:00'),
  'address': 6,
  'item': 26},
 {'id': 16,
  'red': 2,
  'blue': 10,
  'green': 9,
  'orderdate': Timestamp('2020-01-26 17:20:03'),
  'address': 5,
  'item': 21},
 {'id': 19,
  'red': 3,
  'blue': 8,
  'green': 10,
  'orderdate': Timestamp('2020-01-26 17:20:06'),
  'address': 4,
  'item': 21},
 {'id': 23,
  'red': 9,
  'blue': 9,
  'green': 10,
  'orderdate': Timestamp('2020-01-26 17:20:10'),
  'address': 4,
  'item': 28},
 {'id': 124,
  'red': 22,
  'blue': 18,
  'green': 4,
  'orderdate': Timestamp('2020-01-28 18:47:04'),
  'address': 5,
  'item': 44},
 {'id': 125,
  'red': 22,
  'blue': 4,
  'green': 7,
  'orderdate': Timestamp('2020-01-28 18:47:05'),
  'address': 3,
  'item': 33},
 {'id': 126,
  'red': 6,
  'blue': 25,
  'green': 12,
  'orderdate': Timest

In [174]:
def split_order_once(order, item_limit):
    num_partial = np.ceil(order['item'] / item_limit)
    new_order = order.copy()
    new_order['red'] = int(order['red'] // num_partial)
    new_order['green'] = int(order['green'] // num_partial)
    new_order['blue'] = int(order['blue'] // num_partial)
    new_order['item'] = new_order['red'] + new_order['blue'] + new_order['green']
    remaining_order = diff(order.copy(), new_order)
    return [new_order, remaining_order]

In [175]:
def diff(order, new_order):
    order['red'] -= new_order['red']
    order['green'] -= new_order['green']
    order['blue'] -= new_order['blue']
    order['item'] -= new_order['item']
    return order

In [196]:
def split_order_n(order, item_limit):
    partial_orders = [order]
    while count_items(partial_orders[-1]) > item_limit:
        split_order = split_order_once(partial_orders[-1], item_limit)
        partial_orders.pop()
        partial_orders += split_order
    assert order['item'] == count_items(partial_orders)
    for i, partial_order in enumerate(partial_orders, 1):
        partial_order['partialid'] = i
        partial_order['value'] = 1 / len(partial_orders)
    return partial_orders